In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

import os


from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/newdata

/content/drive/My Drive/newdata


In [7]:
df= pd.read_csv('/content/drive/MyDrive/newdata/df_1year.csv')
df_ml = df.copy(())

In [10]:
df_ml.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X58,X59,X60,X61,X62,X63,X64,X65,X66,Y
0,0,-0.991879,0.0,-0.712580,-0.454752,-0.325321,-0.171294,-0.108847,-4.526713,-0.703482,...,0.391845,-0.702995,-0.080575,0.106290,-0.110519,-0.328346,0.768757,-0.438757,-0.470365,0.0
1,1,-0.991160,0.0,0.923473,-0.016287,-0.086800,-0.108155,0.035206,0.000013,0.714048,...,-0.159901,0.734275,0.015761,-0.094074,0.893574,0.567708,-0.451052,0.776698,0.669254,0.0
2,2,-0.991001,0.0,-0.267977,0.695227,-0.463677,-0.327652,-0.885145,0.000014,-0.246794,...,-0.331814,-0.062262,0.269575,1.912255,-0.373765,0.023933,0.933039,-0.489358,0.025104,0.0
3,3,-0.990921,0.0,-0.615815,-0.048014,-1.642791,-0.934659,-2.847291,0.683303,-0.620144,...,-0.480784,-0.657925,0.493823,-0.050944,3.837996,0.522604,3.123654,-0.781235,-0.545474,0.0
4,4,-0.990362,0.0,-0.494688,0.062168,-0.749432,-0.489233,24.786277,-0.000019,-0.514744,...,-0.740004,-0.543087,0.740231,0.485494,0.348445,4.100346,-0.332498,0.486045,-0.030775,0.0


In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components=30)

In [12]:
train = df_ml

train_labels = train['Y']
train = train.drop(columns=['Y','Unnamed: 0','X1','X2'])

X = train
Y = train_labels
feat_names = list(X.columns)

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2,
                                                    random_state=0)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, train_size = 0.9, test_size=0.1, random_state=15)
X_train = pca.fit_transform(X_train)
X_test = pca.fit_transform(X_test)



In [17]:
pip install optuna


     |████████████████████████████████| 308 kB 5.4 MB/s 
     |████████████████████████████████| 81 kB 6.7 MB/s 
     |████████████████████████████████| 210 kB 44.8 MB/s 
     |████████████████████████████████| 78 kB 4.9 MB/s 
     |████████████████████████████████| 146 kB 35.4 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 112 kB 62.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=f5821dd1a4a3336d3a59652866b7ec34e326a3557a7f5a06c20923d9895ba4ff
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [18]:
import optuna
from sklearn import model_selection
import sklearn.metrics as metrics
from xgboost import XGBClassifier
import xgboost as xgb

In [13]:
def objective(trial):
    dtrain = xgb.DMatrix(X_train, label=Y_train)
    dvalid = xgb.DMatrix(X_test, label=Y_test)

    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.f1_score(Y_test, pred_labels,average ='weighted')
    return accuracy

In [14]:
import sklearn

In [19]:
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), direction="maximize")
study.optimize(objective, n_trials=1000)

[I 2022-05-14 12:51:34,041] A new study created in memory with name: no-name-5756899a-0985-4e11-90e0-560aaa52fbec


[0]	validation-auc:0.413802
[1]	validation-auc:0.413986
[2]	validation-auc:0.414071
[3]	validation-auc:0.414043
[4]	validation-auc:0.414043
[5]	validation-auc:0.414043
[6]	validation-auc:0.414029
[7]	validation-auc:0.414029
[8]	validation-auc:0.438096
[9]	validation-auc:0.442308


[I 2022-05-14 12:51:34,406] Trial 0 finished with value: 0.9138654418120996 and parameters: {'booster': 'gbtree', 'lambda': 8.391618948061256e-07, 'alpha': 0.031523903142261, 'max_depth': 9, 'eta': 0.00452507080239743, 'gamma': 8.803441823030175e-07, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 0.9138654418120996.


[0]	validation-auc:0.5
[1]	validation-auc:0.5
[2]	validation-auc:0.5
[3]	validation-auc:0.5
[4]	validation-auc:0.5
[5]	validation-auc:0.5
[6]	validation-auc:0.491349
[7]	validation-auc:0.5
[8]	validation-auc:0.5
[9]	validation-auc:0.5


[I 2022-05-14 12:51:34,504] Trial 1 finished with value: 0.9384005819531372 and parameters: {'booster': 'gbtree', 'lambda': 0.00022463239777711276, 'alpha': 2.9759360837535223e-05, 'max_depth': 1, 'eta': 1.4153765039127997e-08, 'gamma': 3.870273274727505e-07, 'grow_policy': 'lossguide'}. Best is trial 1 with value: 0.9384005819531372.


...

[I 2022-05-14 12:56:14,000] Trial 997 finished with value: 0.9386642093558477 and parameters: {'booster': 'gbtree', 'lambda': 3.3286396087733706e-08, 'alpha': 0.00046743416103030435, 'max_depth': 3, 'eta': 7.665357301849759e-07, 'gamma': 0.00011434244403038254, 'grow_policy': 'lossguide'}. Best is trial 20 with value: 0.9386642093558477.


[0]	validation-auc:0.498213
[1]	validation-auc:0.498213
[2]	validation-auc:0.480769
[3]	validation-auc:0.498213
[4]	validation-auc:0.480471
[5]	validation-auc:0.480471
[6]	validation-auc:0.480471
[7]	validation-auc:0.480471
[8]	validation-auc:0.480471
[9]	validation-auc:0.480471


[I 2022-05-14 12:56:14,433] Trial 998 finished with value: 0.9386642093558477 and parameters: {'booster': 'gbtree', 'lambda': 3.151734659306543e-07, 'alpha': 2.131068208028094e-05, 'max_depth': 3, 'eta': 3.686889020187688e-07, 'gamma': 0.03730028810387839, 'grow_policy': 'depthwise'}. Best is trial 20 with value: 0.9386642093558477.


[0]	validation-auc:0.498213
[1]	validation-auc:0.498213
[2]	validation-auc:0.480471
[3]	validation-auc:0.480471
[4]	validation-auc:0.480471
[5]	validation-auc:0.480471
[6]	validation-auc:0.480471
[7]	validation-auc:0.480471
[8]	validation-auc:0.480471
[9]	validation-auc:0.480471


[I 2022-05-14 12:56:14,820] Trial 999 finished with value: 0.9386642093558477 and parameters: {'booster': 'gbtree', 'lambda': 9.350828803034047e-08, 'alpha': 1.7550667144931726e-05, 'max_depth': 3, 'eta': 6.135530000728981e-07, 'gamma': 0.06696361625633403, 'grow_policy': 'depthwise'}. Best is trial 20 with value: 0.9386642093558477.


In [20]:
params_2 = study.best_params

In [21]:
params_2

{'alpha': 0.004238227171953563,
 'booster': 'gbtree',
 'eta': 1.745749442748138e-07,
 'gamma': 0.0059789004893609,
 'grow_policy': 'lossguide',
 'lambda': 0.0004996753261218628,
 'max_depth': 3}

In [22]:
clf2 = XGBClassifier(**params_2)
clf2.fit(X_train, Y_train, verbose=True, early_stopping_rounds=1000, eval_set=[[X_test,Y_test]])

[0]	validation_0-error:0.045599
Will train until validation_0-error hasn't improved in 1000 rounds.
[1]	validation_0-error:0.042418
[2]	validation_0-error:0.042418
[3]	validation_0-error:0.042418
[4]	validation_0-error:0.041357
[5]	validation_0-error:0.041357
[6]	validation_0-error:0.042418
[7]	validation_0-error:0.041357
[8]	validation_0-error:0.041357
[9]	validation_0-error:0.041357
[10]	validation_0-error:0.041357
[11]	validation_0-error:0.041357
[12]	validation_0-error:0.041357
[13]	validation_0-error:0.041357
[14]	validation_0-error:0.041357
[15]	validation_0-error:0.041357
[16]	validation_0-error:0.041357
[17]	validation_0-error:0.041357
[18]	validation_0-error:0.041357
[19]	validation_0-error:0.041357
[20]	validation_0-error:0.041357
[21]	validation_0-error:0.041357
[22]	validation_0-error:0.041357
[23]	validation_0-error:0.041357
[24]	validation_0-error:0.041357
[25]	validation_0-error:0.041357
[26]	validation_0-error:0.041357
[27]	validation_0-error:0.041357
[28]	validation_0-

XGBClassifier(alpha=0.004238227171953563, eta=1.745749442748138e-07,
              gamma=0.0059789004893609, grow_policy='lossguide',
              lambda=0.0004996753261218628)

In [23]:
score2 = clf2.score(X_test, Y_test)
pred_proba_t2 = clf2.predict_proba(X_test)


y_pred2 = clf2.predict(X_test)

In [24]:
print(classification_report(Y_test, y_pred2))


              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       904
         1.0       0.00      0.00      0.00        39

    accuracy                           0.96       943
   macro avg       0.48      0.50      0.49       943
weighted avg       0.92      0.96      0.94       943



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
